In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('freMTPL2freq.csv', sep=';', decimal = ',')


In [3]:
# Pre-process features
data['VehPower'] = np.log(data['VehPower'])

bins = [0, 6, 13, float('inf')]  
labels = ['smallest', 'mid', 'largest'] 
data['VehAge'] = pd.cut(data['VehAge'], bins=bins, labels=labels, right=False)

data['Log_DrivAge'] = np.log(data['DrivAge'])
data['Log_BonusMalus'] = np.log(data['BonusMalus'])
data['Log_Density'] = np.log(data['Density'])

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
ds_ct_variables = ['VehPower', 'DrivAge', 'BonusMalus', 'Density', ]
X = data.drop(columns = ['ClaimNb', 'Exposure'])
X = pd.get_dummies(X, drop_first=True)
Y = data['ClaimNb'] / data['Exposure']
Exposure = data['Exposure']

# Train-test split
X_train, X_test, Y_train, Y_test, exposure_train, exposure_test = train_test_split( 
    X, Y, Exposure, test_size=0.1
)

X_train[ds_ct_variables] = scaler.fit_transform(X_train[ds_ct_variables])
X_test[ds_ct_variables] = scaler.fit_transform(X_test[ds_ct_variables])


model = PoissonRegressor(alpha = 0)
model.fit(X_train,Y_train, sample_weight=exposure_train)

# Print MAE, MSE and loss on train and test data sets

Y_pred_train = model.predict(X_train)
Y_pred_test = model.predict(X_test)

mae_train = mean_absolute_error(Y_train*exposure_train, Y_pred_train)
mae_test = mean_absolute_error(Y_test*exposure_test, Y_pred_test)

mse_train = mean_squared_error(Y_train*exposure_train, Y_pred_train)
mse_test = mean_squared_error(Y_test*exposure_test, Y_pred_test)

A=np.log(Y_pred_train/Y_train)
B=np.log(Y_pred_test / Y_test)

loss_train = np.sum(exposure_train * 2*(Y_pred_train - Y_train - Y_train * A)) / np.sum(exposure_train)
loss_test = np.sum(exposure_test * 2*(Y_pred_test - Y_test - Y_test * B)) / np.sum(exposure_test)

print(f"Train Loss: {loss_train}, Train MAE: {mae_train}, Train MSE: {mse_train}")
print(f"Test Loss: {loss_test}, Test MAE: {mae_test}, Test MSE: {mse_test}")

Train Loss: 0.3196100097097818, Train MAE: 0.1100268541911783, Train MSE: 0.04469596952596384
Test Loss: 0.31591100995787774, Test MAE: 0.10932676366083637, Test MSE: 0.043524846299371193


In [4]:
# Neural network implementation
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from tqdm import tqdm

nh1 = 64
nh2 = 64
nh3 = 32

n_epochs1 = 20
n_epochs2 = 100
nbfolds=5
nbmodels = 4
lr = [1e-3,1e-3,1e-2,1e-2]
w= [1e-2, 1e-3, 1e-2, 1e-3]
batch_size = 10000
        

class CustomNetwork(torch.nn.Module):
    def __init__(self, input_size, nh1, nh2, nh3):
        super(CustomNetwork, self).__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(input_size, nh1),
            torch.nn.ReLU(),
            torch.nn.Linear(nh1, nh2),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(nh2, nh3),
            torch.nn.ReLU(),
            torch.nn.Linear(nh3, 1),
        )

    def forward(self, x):
        x = self.layers(x)
        x=torch.clamp(x, max=1e10) #x can be at max 20 so that then the exponential doesn't explode (especially at the first iterations)
        return torch.exp(x)  # Apply the exponential activation here



#Loss function 
def poisson_loss(y_pred, y_true, exposure):

    term1 = y_pred  # λ̂
    term2 = y_true  # y
    term3 = y_true * torch.log(y_pred+1e-10)  # y * log(λ̂), we add 1e-10 for numerical stability in case 0 is predicted
    #it can happen to predict 0 if the last linear layer predicts a very negative number such as 1e-20, then exp of that will be seen as 0 by the computer
    term4 = torch.where(y_true > 0, y_true * torch.log(y_true), torch.zeros_like(y_true))  # y * log(y), 0 if y = 0
    loss = 2 * (term1 - term2 - term3 + term4)
    weighted_loss = torch.sum(exposure * loss) / torch.sum(exposure)  # Exposure-weighted loss

    return weighted_loss


# We do 5 fold cross validation to select the set of hyperparameters for our model (learning rate and weight decay)

model_loss = []

kf = KFold(n_splits=nbfolds, shuffle=True, random_state=42)

for nbr in range(nbmodels):
    print(f"Starting model {nbr + 1}...")
    fold_loss = []
    for fold, (train_index, val_index) in enumerate(kf.split(X_train)):

        #initialize network and optimizer for each fold and set of hyperparameters
        network = CustomNetwork(X_train.shape[1], nh1, nh2, nh3)
        optimizer = torch.optim.Adam(network.parameters(), lr=lr[nbr], weight_decay=w[nbr])
        
        Xftrain, Xftest = X_train.iloc[train_index], X_train.iloc[val_index]
        Yftrain, Yftest = Y_train.iloc[train_index.tolist()], Y_train.iloc[val_index.tolist()]
        expftrain, expftest = exposure_train.iloc[train_index.tolist()], exposure_train.iloc[val_index.tolist()]
        
        X_train_torch=torch.tensor(Xftrain.astype(float).values, dtype=torch.float32)
        Y_train_torch=torch.tensor(Yftrain.astype(float).values, dtype=torch.float32).view(-1,1)
        X_test_torch=torch.tensor(Xftest.astype(float).values, dtype=torch.float32)
        Y_test_torch=torch.tensor(Yftest.astype(float).values, dtype=torch.float32).view(-1,1)
        exposure_train_torch=torch.tensor(expftrain.astype(float).values, dtype=torch.float32).view(-1,1)
        exposure_test_torch=torch.tensor(expftest.astype(float).values, dtype=torch.float32).view(-1,1)
    
        train_dataset = TensorDataset(X_train_torch, Y_train_torch, exposure_train_torch)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, persistent_workers=True)
    
        training_loop = tqdm(range(n_epochs1)) 
        
        for epoch in training_loop:
            epoch_loss = 0
            network.train()
            for (x,y,e) in train_loader:
    
                optimizer.zero_grad()
                batch_loss = poisson_loss(network(x), y, e)
                batch_loss.backward()
                optimizer.step()
                
                epoch_loss += batch_loss.item()
        
            # Average loss per batch
            epoch_loss /= len(train_loader)
            training_loop.set_postfix(loss=epoch_loss)
            
         # Validation loop
        # Calculate loss for the fold
        network.eval()
        with torch.no_grad():
            loss=poisson_loss(network(X_test_torch), Y_test_torch, exposure_test_torch)
        
        fold_loss.append(loss.item())
    #average the fold losses for each model
    model_loss.append(np.sum(fold_loss)/nbfolds)

Starting model 1...


100%|██████████| 20/20 [00:31<00:00,  1.58s/it, loss=0.466]


Starting model 2...


100%|██████████| 20/20 [00:31<00:00,  1.57s/it, loss=0.459]


Starting model 3...


100%|██████████| 20/20 [00:32<00:00,  1.61s/it, loss=0.463]


Starting model 4...


100%|██████████| 20/20 [00:31<00:00,  1.58s/it, loss=0.455]


In [5]:
print(model_loss)
minval=min(model_loss)
mind=model_loss.index(minval)
print(f"The best model is the number {mind + 1} with learning rate {lr[mind]} and L2 weight {w[mind]}")

[0.4588212728500366, 0.45560272932052615, 0.4595827341079712, 0.45455277562141416]
The best model is the number 4 with learning rate 0.01 and L2 weight 0.001


In [6]:
#We train our model on the full train data using the best hyperparameters chosen by cross validation
n_epochs2 = 100
network = CustomNetwork(X_train.shape[1], nh1, nh2, nh3)
opt1 = torch.optim.Adam(network.parameters(), lr=lr[mind], weight_decay=w[mind])

X_train_torch=torch.tensor(X_train.astype(float).values, dtype=torch.float32)
Y_train_torch=torch.tensor(Y_train.astype(float).values, dtype=torch.float32).view(-1,1)
X_test_torch=torch.tensor(X_test.astype(float).values, dtype=torch.float32)
Y_test_torch=torch.tensor(Y_test.astype(float).values, dtype=torch.float32).view(-1,1)
exposure_train_torch=torch.tensor(exposure_train.astype(float).values, dtype=torch.float32).view(-1,1)
exposure_test_torch=torch.tensor(exposure_test.astype(float).values, dtype=torch.float32).view(-1,1)
    
train_dataset = TensorDataset(X_train_torch, Y_train_torch, exposure_train_torch)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, persistent_workers=True)
    
training_loop = tqdm(range(n_epochs2)) 
        
for epoch in training_loop:
    epoch_loss = 0
    network.train()
    for (x,y,e) in train_loader:
    
        opt1.zero_grad()
        batch_loss = poisson_loss(network(x), y, e)
        batch_loss.backward()
        opt1.step()
                
        epoch_loss += batch_loss.item()
        
    # Average loss per batch
    epoch_loss /= len(train_loader)
    training_loop.set_postfix(loss=epoch_loss)
            
network.eval()
with torch.no_grad():
    train_loss=poisson_loss(network(X_train_torch), Y_train_torch, exposure_train_torch)
    test_loss=poisson_loss(network(X_test_torch), Y_test_torch, exposure_test_torch)
    mae_train=mean_absolute_error(Y_train*exposure_train, network(X_train_torch))
    mae_test=mean_absolute_error(Y_test*exposure_test, network(X_test_torch))
    mse_train=mean_squared_error(Y_train*exposure_train, network(X_train_torch))
    mse_test=mean_squared_error(Y_test*exposure_test, network(X_test_torch))

100%|██████████| 100/100 [04:44<00:00,  2.85s/it, loss=0.455]


In [7]:
print(f"Train Loss: {train_loss}, Train MAE: {mae_train}, Train MSE: {mse_train}")
print(f"Test Loss: {test_loss}, Test MAE: {mae_test}, Test MSE: {mse_test}")

Train Loss: 0.4556354284286499, Train MAE: 0.12242735336229696, Train MSE: 0.04531353369204714
Test Loss: 0.455142617225647, Test MAE: 0.12240879695199187, Test MSE: 0.045381144854904874


In [8]:
# Implement a regression tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

tree_model = DecisionTreeRegressor(random_state=42)
param_grid = {'min_impurity_decrease': [0, 0.01, 0.05, 0.1, 0.2]}
grid_search = GridSearchCV(tree_model, param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
grid_search.fit(X_train, Y_train, sample_weight=exposure_train)
best_tree_model = grid_search.best_estimator_

# Cross-validation
Y_pred_train_tree = best_tree_model.predict(X_train)
Y_pred_test_tree = best_tree_model.predict(X_test)

mae_train_tree = mean_absolute_error(Y_train * exposure_train, Y_pred_train_tree)
mae_test_tree = mean_absolute_error(Y_test * exposure_test, Y_pred_test_tree)
mse_train_tree = mean_squared_error(Y_train * exposure_train, Y_pred_train_tree)
mse_test_tree = mean_squared_error(Y_test * exposure_test, Y_pred_test_tree)

A_tree = np.log(Y_pred_train_tree / Y_train)
A_tree[np.isinf(A_tree)] = 0
B_tree = np.log(Y_pred_test_tree / Y_test)
B_tree[np.isinf(B_tree)] = 0

loss_train_tree = np.sum(exposure_train * 2 * (Y_pred_train_tree - Y_train - Y_train * A_tree)) / np.sum(exposure_train)
loss_test_tree = np.sum(exposure_test * 2 * (Y_pred_test_tree - Y_test - Y_test * B_tree)) / np.sum(exposure_test)

# Print MAE, MSE and loss on train and test data sets
print("Decision Tree Results:")
print(f"Best min_impurity_decrease: {grid_search.best_params_['min_impurity_decrease']}")
print(f"Train Loss: {loss_train_tree}, Train MAE: {mae_train_tree}, Train MSE: {mse_train_tree}")
print(f"Test Loss: {loss_test_tree}, Test MAE: {mae_test_tree}, Test MSE: {mse_test_tree}")

Decision Tree Results:
Best min_impurity_decrease: 0.01
Train Loss: 0.47755958040568236, Train MAE: 0.10711256181518547, Train MSE: 0.043123485555150075
Test Loss: 0.4766356259477309, Test MAE: 0.10706989794660637, Test MSE: 0.04316239876776954


In [9]:
# Implement a random forest model
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(random_state=42)
param_grid = {'min_impurity_decrease': [0, 0.01, 0.05],'max_features': ['sqrt', 'log2', 0.8]}  
grid_search = GridSearchCV(rf_model, param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
grid_search.fit(X_train, Y_train, sample_weight=exposure_train)
best_rf_model = grid_search.best_estimator_

# Cross-validation
Y_pred_train_rf = best_rf_model.predict(X_train)
Y_pred_test_rf = best_rf_model.predict(X_test)

mae_train_rf = mean_absolute_error(Y_train * exposure_train, Y_pred_train_rf)
mae_test_rf = mean_absolute_error(Y_test * exposure_test, Y_pred_test_rf)

mse_train_rf = mean_squared_error(Y_train * exposure_train, Y_pred_train_rf)
mse_test_rf = mean_squared_error(Y_test * exposure_test, Y_pred_test_rf)

A_rf = np.log(Y_pred_train_rf / Y_train)
A_rf[np.isinf(A_rf)] = 0
B_rf = np.log(Y_pred_test_rf / Y_test)
B_rf[np.isinf(B_rf)] = 0

loss_train_rf = np.sum(exposure_train * 2 * (Y_pred_train_rf - Y_train - Y_train * A_rf)) / np.sum(exposure_train)
loss_test_rf = np.sum(exposure_test * 2 * (Y_pred_test_rf - Y_test - Y_test * B_rf)) / np.sum(exposure_test)

# Print MAE, MSE and loss on train and test data sets
print("Random Forest Results:")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Train Loss: {loss_train_rf}, Train MAE: {mae_train_rf}, Train MSE: {mse_train_rf}")
print(f"Test Loss: {loss_test_rf}, Test MAE: {mae_test_rf}, Test MSE: {mse_test_rf}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Random Forest Results:
Best Parameters: {'max_features': 'sqrt', 'min_impurity_decrease': 0.01}
Train Loss: 0.4775595808183377, Train MAE: 0.10711766784912458, Train MSE: 0.04312386800791132
Test Loss: 0.4766356098079722, Test MAE: 0.10707500484963621, Test MSE: 0.04316278181876905


In [10]:
# Implement gradient boosted trees
from sklearn.ensemble import GradientBoostingRegressor

gbt_model = GradientBoostingRegressor(random_state=42)
param_grid = {'learning_rate': [0.01, 0.1, 0.2],'n_estimators': [100, 200, 300]}
grid_search = GridSearchCV(gbt_model, param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
grid_search.fit(X_train, Y_train, sample_weight=exposure_train)
best_gbt_model = grid_search.best_estimator_

# Cross-validation
Y_pred_train_gbt = np.clip(best_gbt_model.predict(X_train), 1e-10, None)
Y_pred_test_gbt = np.clip(best_gbt_model.predict(X_test), 1e-10, None)

mae_train_gbt = mean_absolute_error(Y_train * exposure_train, Y_pred_train_gbt)
mae_test_gbt = mean_absolute_error(Y_test * exposure_test, Y_pred_test_gbt)

mse_train_gbt = mean_squared_error(Y_train * exposure_train, Y_pred_train_gbt)
mse_test_gbt = mean_squared_error(Y_test * exposure_test, Y_pred_test_gbt)

A_gbt = np.log((Y_pred_train_gbt + 1e-10) / (Y_train + 1e-10))
A_gbt[np.isinf(A_gbt)] = 0
B_gbt = np.log((Y_pred_test_gbt + 1e-10) / (Y_test + 1e-10))
B_gbt[np.isinf(B_gbt)] = 0

loss_train_gbt = np.sum(exposure_train * 2 * (Y_pred_train_gbt - Y_train - Y_train * A_gbt)) / np.sum(exposure_train)
loss_test_gbt = np.sum(exposure_test * 2 * (Y_pred_test_gbt - Y_test - Y_test * B_gbt)) / np.sum(exposure_test)


# Print MAE, MSE and loss on train and test data sets
print("Gradient Boosted Trees Results:")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Train Loss: {loss_train_gbt}, Train MAE: {mae_train_gbt}, Train MSE: {mse_train_gbt}")
print(f"Test Loss: {loss_test_gbt}, Test MAE: {mae_test_gbt}, Test MSE: {mse_test_gbt}")

Gradient Boosted Trees Results:
Best Parameters: {'learning_rate': 0.1, 'n_estimators': 300}
Train Loss: 0.4434741915445735, Train MAE: 0.10884477467562399, Train MSE: 0.044344323516349406
Test Loss: 0.4463857177430613, Test MAE: 0.108792878544363, Test MSE: 0.04407424275252587
